In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from integrators.pod.client import PodClient
from integrators.data.basic import *
from integrators.importers.email import EmailImporter, DEFAULT_GMAIL_HOST
from integrators.importers.util import *
from integrators.imports import *
from integrators.data.schema import *
from io import BufferedReader, BytesIO
from time import sleep

import os
import signal
import subprocess
import requests
import time

from selenium.webdriver import Chrome, ChromeOptions
from webdriver_manager.chrome import ChromeDriverManager

# Login

In [ ]:
class CVATClient():
    
    def __init__(self, username="admin", password="admin", url="http://localhost", port=8080):
        self.base_url = f"{url}:{port}/api/v1"
        self.username=username
        self.password=password
        self.session = requests.Session()
        token = self.login(username, password)
        self.session.headers.update({"Authorization": f"Token {token}",
                                     "Accept": "application/json"})

    def login(self, username, password):
        data = {"username": username, "password": password}
        res = self.session.post(f"{self.base_url}/auth/login", json=data)
        res.raise_for_status()
        return res.json()["key"]
    
    def create_task(self, labels):
        """creates task by defining labels"""
        res = self.session.post(f"{self.base_url}/tasks", json=labels)
        res.raise_for_status()
        return res.json()["id"]
    
    def assert_upload_succes(self, task_id):
        res = self.session.get(f"{self.base_url}/tasks/{task_id}/data/meta")
        assert len(res.json()["frames"]) > 0 
        
    def get_job(self, task_id):
        res = self.session.get(f"{self.base_url}/tasks/{task_id}/jobs")
        res.raise_for_status()
        return res.json()[0]["id"]
        
    
    def upload_data(self, task_id, file_buffers):
        """filebuffers of photos"""
        files = {f"client_files[{i}]": f for i, f in enumerate(file_buffers)}
        res = self.session.post(f"{self.base_url}/tasks/{task_id}/data",
                           files=files,
                           data={"image_quality": 70})
        res.raise_for_status()
        sleep(1)
        self.assert_upload_succes(task_id)
        # uploading data for a task automatically creates a "job", which we need to start annotating
        # so we fetch it here and return
        return self.get_job(task_id)
    

In [ ]:
c_client = CVATClient()

# Define labels & Create project

In [ ]:
labels = {"name":"test",
          "labels":[{"name":"label1", "attributes": []},
                    {"name":"label2", "attributes": [], "color":"#7c0e99"}]}

In [ ]:
task_id = c_client.create_task(labels)

# Add data

When we add data, we automatically generate a job. We need this job id to go to the right url

In [ ]:
folder = PYI_TESTDATA / "photos" / "facerecognition"
paths = [folder / "ellen1.png", folder / "celebs.jpg"]
file_buffers = [open(p, "rb") for p in paths]
job_id = c_client.upload_data(task_id, file_buffers)

## Open labeling

In [ ]:
options = ChromeOptions()
options.add_argument("--kiosk")
driver = Chrome(ChromeDriverManager().install())
driver.maximize_window()
driver.get(f"http://localhost:8080")
username = driver.find_element_by_id("username")
pas = driver.find_element_by_id("password")
button = driver.find_element_by_class_name("login-form-button")
username.send_keys("admin")
pas.send_keys("admin")
button.click()
sleep(1)
driver.get(f"http://localhost:8080/tasks/{task_id}/jobs/{job_id}")

[WDM] - Current google-chrome version is 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324


[WDM] - Get LATEST driver version for 88.0.4324
[WDM] - Trying to download new driver from http://chromedriver.storage.googleapis.com/88.0.4324.96/chromedriver_mac64.zip
[WDM] - Driver has been saved in cache [/Users/koen/.wdm/drivers/chromedriver/mac64/88.0.4324.96]


## Apendix

In [ ]:
# def generate_image_file(filename, size=(100, 50)):
#     f = BytesIO()
#     image = Image.new('RGB', size=size)
#     image.save(f, 'jpeg')
#     f.name = filename
#     f.seek(0)
#     return f

In [ ]:
# import zipfile
# from PIL import Image

In [ ]:
# paths = "test.jpg"
# f = BytesIO()
# with zipfile.ZipFile(f, 'w') as archive:
#     for path in paths:
#         archive.writestr(path, generate_image_file(path).getvalue())
# f.name = 'images.zip'
# f.seek(0)

# files= {'client_files[0]': f}

In [ ]:
base_url = "http://localhost:8080/api/v1"

In [ ]:
session = requests.Session()
data = {"username": "admin", "password": "admin"}
res = session.post(f"{base_url}/auth/login", json=data)
res.raise_for_status()
token =  res.json()["key"]
session.headers.update({"Authorization": f"Token {token}",
                             "Accept": "application/json"})

In [ ]:
res = session.post(f"{base_url}/tasks", json=labels)
res.raise_for_status()
task_id = res.json()["id"]

In [ ]:
task_id

51

In [ ]:
folder = PYI_TESTDATA / "photos" / "facerecognition"

In [ ]:
paths = [folder / "ellen1.png", folder / "celebs.jpg"]
files = {f"client_files[{i}]": open(p, "rb") for i, p in enumerate(paths)}
# files = {"client_files[0]": open(folder / "ellen1.png", "rb")}
res = session.post(f"{base_url}/tasks/{task_id}/data",
                   files=files,
                   data={"image_quality": 70})

In [ ]:
res = session.get(f"{base_url}/tasks/{52}/data/meta")


In [ ]:
res.json()["frames"]

[{'width': 1920, 'height': 1080, 'name': 'celebs.jpg'},
 {'width': 112, 'height': 112, 'name': 'ellen1.png'}]

In [ ]:
res = c_client.session.get(f"{base_url}/tasks/{52}/data/meta")